In [235]:
import os
import csv
import pandas as pd
import numpy as np

csvpath = os.path.join('.','2016_Annual_Crime_Itemized_Data_ATX.csv')

#print(f'csvpath: {csvpath}')


In [236]:
with open(csvpath, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    csvheader = next(csvreader)
    print(f'CSV Header: {csvheader}')
#    for row in csvreader:
#       print(row)

CSV Header: ['ï»¿GO Primary Key', 'Council District', 'GO Highest Offense Desc', 'Highest NIBRS/UCR Offense Description', 'GO Report Date', 'GO Location', 'Clearance Status', 'Clearance Date', 'GO District', 'GO Location Zip', 'GO Census Tract', 'GO X Coordinate', 'GO Y Coordinate']


In [237]:
atx_crime_info_df = pd.read_csv(csvpath)
atx_crime_info_df.head()

,GO Primary Key,Council District,GO Highest Offense Desc,Highest NIBRS/UCR Offense Description,GO Report Date,GO Location,Clearance Status,Clearance Date,GO District,GO Location Zip,GO Census Tract,GO X Coordinate,GO Y Coordinate
0,"201,610,188",8.0,AGG ASLT ENHANC STRANGL/SUFFOC,Agg Assault,1-Jan-16,8600 W SH 71 ...,C,12-Jan-16,D,78735.0,19.08,"3,067,322","10,062,796"
1,"201,610,643",9.0,THEFT,Theft,1-Jan-16,219 E 6TH ST ...,C,4-Jan-16,G,78701.0,11,"3,114,957","10,070,462"
2,"201,610,892",4.0,AGG ROBBERY/DEADLY WEAPON,Robbery,1-Jan-16,701 W LONGSPUR BLVD ...,N,3-May-16,E,78753.0,18.23,"3,129,181","10,106,923"
3,"201,610,893",9.0,THEFT,Theft,1-Jan-16,404 COLORADO ST ...,N,22-Jan-16,G,78701.0,11,"3,113,643","10,070,357"
4,"201,611,018",4.0,SEXUAL ASSAULT W/ OBJECT,Rape,1-Jan-16,NaN,C,10-Mar-16,E,78753.0,18.33,NaN,NaN


In [238]:
#cols = ['GO Primary Key', 'GO Location Zip', 'GO Census Tract', 'Highest NIBRS/UCR Offense Description', ]

In [239]:
#atx_crime_info_df.filter(regex='^GO')

# select the needed columns
#   GO Location Zip : the zipcode that the incident happened in
#   GO Census Tract : the census tract where the incdent occurried

crime_df = atx_crime_info_df[['GO Primary Key', 'GO Location Zip', 'Highest NIBRS/UCR Offense Description']]

crime_df.head()

,GO Primary Key,GO Location Zip,Highest NIBRS/UCR Offense Description
0,"201,610,188",78735.0,Agg Assault
1,"201,610,643",78701.0,Theft
2,"201,610,892",78753.0,Robbery
3,"201,610,893",78701.0,Theft
4,"201,611,018",78753.0,Rape


In [240]:
#Rename columns
crime_df = crime_df.rename(columns = { 'GO Location Zip' : 'Zipcode',
                                       'GO Primary Key': 'Cnt',
                                      # 'GO Census Tract' : 'Census Tract', 
                                       'Highest NIBRS/UCR Offense Description' : 'Category'})
crime_df = crime_df.dropna(how='any') 
crime_df.head()

,Cnt,Zipcode,Category
0,"201,610,188",78735.0,Agg Assault
1,"201,610,643",78701.0,Theft
2,"201,610,892",78753.0,Robbery
3,"201,610,893",78701.0,Theft
4,"201,611,018",78753.0,Rape


In [242]:
crime_df["Zipcode"] = crime_df['Zipcode'].astype(int)
crime_df['Cnt'] = crime_df['Cnt'] = 1
crime_df = crime_df.drop(['Category'],axis = 1)
crime_df.head()

,Cnt,Zipcode
0,1,78735
1,1,78701
2,1,78753
3,1,78701
4,1,78753


In [243]:
crime_grp_by = crime_df.groupby(by="Zipcode")
crime_grp_by.count().head(100).max()
max_rating = crime_grp_by.size().max()
# also known as the max rating cnt
max_rating

3355

In [244]:
crime_grp_by.count().head(10)

,Cnt
Zipcode,
78613,341
78617,285
78652,16
78653,48
78660,179
78664,2
78701,2076
78702,1582
78703,660


In [245]:
outpath = os.path.join('..','2016_ATX_Crime_Cnts.csv')
outpath

'..\\2016_ATX_Crime_Cnts.csv'

In [246]:
crime_grp_by.count().to_csv(outpath)

In [247]:
# re-load the output file as a dataframe
crime_cnts_df = pd.read_csv(outpath)
crime_cnts_df.head()

,Zipcode,Cnt
0,78613,341
1,78617,285
2,78652,16
3,78653,48
4,78660,179


In [248]:
# calculate a 'Rating' by dividing zipcode count by the max rating cnt
crime_cnts_df['Rating'] = (crime_cnts_df.Cnt / max_rating) * 10
#crime_cnts_df.max()

Zipcode    78759.0
Cnt         3355.0
Rating        10.0
dtype: float64

In [249]:
outratings = os.path.join('..','2016_ATX_Crime_Ratings.csv')
outratings

'..\\2016_ATX_Crime_Ratings.csv'

In [252]:
#print(crime_cnts_df)

In [251]:
crime_cnts_df.to_csv(outratings)